In [22]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import argparse
import time
import os
from model import LSTM

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [24]:
parser = argparse.ArgumentParser(description='Add these argument for training')
parser.add_argument('--dir', default='results', help='directory for saving trianed mode')
parser.add_argument('--feature', required= True)

parser.add_argument('--lr', default=0.005, help='learning rate')
parser.add_argument('--epochs', default=10, help='epoch number')
parser.add_argument('--batch_size', default=32)
args = parser.parse_args()

## Set parameter for the model
directory = args.dir                # directory for saving model
batch_size = int(args.batch_size)   # batch size
learning_rate = float(args.lr)
num_epochs = int(args.epochs)

### **Data Processing**

In [10]:
data = pd.read_csv('data/solar_wind_parameters_data_1_hourly_all.csv')

data = data.drop(columns=['Unnamed: 0','Timestamp'])

# data.head()

In [ ]:
features = ['Scalar_B', 'BX_GSE_GSM', 'BY_GSE', 'BZ_GSE', 'BY_GSM', 'BZ_GSM', 'Proton_Density', 'SW_Plasma_Temperature', 'SW_Plasma_Speed']
target = ['Dst-index']

# Select the features and target
data = data[features + target]
# data.head()

# Normalize the data
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Split the data into sequences
def sequence(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1]) # all columns except the last
        y.append(data[i+seq_length, -1])  # target is the last column
        # print(f'X:{np.array(X)}, Y:{np.array(y)}')
    return np.array(X), np.array(y)

SEQ_LENGTH = 24
X, y = sequence(data_scaled, SEQ_LENGTH)


# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)
print(f'Train shape: {X_train.shape}, {y_train.shape}')
print(f'Test shape: {X_test.shape}, {y_test.shape}')

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_load = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_load = DataLoader(test_data, batch_size=batch_size, shuffle=False)


"\n# Normalize the data\nfrom sklearn.preprocessing import MinMaxScaler\nfrom sklearn.model_selection import train_test_split\n\nscaler = MinMaxScaler()\ndata_scaled = scaler.fit_transform(data)\n\n# Split the data into sequences\ndef sequence(data, seq_length):\n    X, y = [], []\n    for i in range(len(data) - seq_length):\n        X.append(data[i:i+seq_length, :-1]) # all columns except the last\n        y.append(data[i+seq_length, -1])  # target is the last column\n        # print(f'X:{np.array(X)}, Y:{np.array(y)}')\n    return np.array(X), np.array(y)\n\nSEQ_LENGTH = 24\nX, y = sequence(data_scaled, SEQ_LENGTH)\n\n\n# Convert to PyTorch tensors\nX_tensor = torch.tensor(X, dtype=torch.float32).to(device)\ny_tensor = torch.tensor(y, dtype=torch.float32).to(device)\n\n# Split the data into training and testing sets\nX_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)\nprint(f'Train shape: {X_train.shape}, {y_train.shape}')\nprint(f'T

Load data from Dst-prediction

In [25]:
data = pd.read_csv('data/solar-wind-data/1h/solar_wind_parameters_data_1_hourly_all.csv')
# data = data.drop(columns=['Unnamed: 0','Timestamp'])

features = ['Scalar_B',  'BZ_GSE', 'SW_Plasma_Temperature',  'SW_Proton_Density','SW_Plasma_Speed', 'Flow_pressure', 'E_elecrtric_field']
target = ['Dst-index']

# Select the features and target
data = data[features + target]
data.head()
# print(data.shape)

,Scalar_B,BZ_GSE,SW_Plasma_Temperature,SW_Proton_Density,SW_Plasma_Speed,Flow_pressure,E_elecrtric_field,Dst-index
0,6.5,-3.3,9999999,2.0,430,0.74,1.03,-13
1,6.3,-1.5,145000,2.1,439,0.81,0.26,-14
2,5.7,1.9,131000,1.9,449,0.77,-1.17,-17
3,5.7,2.5,76000,1.6,446,0.64,-1.43,-17
4,5.9,2.9,78000,1.4,468,0.61,-1.54,-16


In [26]:
# Normalize the data
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Split the data into sequences
def sequence(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length, :-1]) # all columns except the last
        y.append(data[i+seq_length, -1])  # target is the last column
        # print(f'X:{np.array(X)}, Y:{np.array(y)}')
    return np.array(X), np.array(y)

SEQ_LENGTH = 24
X, y = sequence(data_scaled, SEQ_LENGTH)


# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).to(device)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, shuffle=False)
print(f'Train shape: {X_train.shape}, {y_train.shape}')
print(f'Test shape: {X_test.shape}, {y_test.shape}')

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_load = DataLoader(train_data, batch_size=batch_size, shuffle=False)
test_load = DataLoader(test_data, batch_size=batch_size, shuffle=False)


Train shape: torch.Size([370771, 24, 7]), torch.Size([370771])
Test shape: torch.Size([92693, 24, 7]), torch.Size([92693])


### **MAIN**

In [29]:

if __name__ == "__main__":
    input_size = len(features)
    hidden_layer_size = 48
    output_size = 1
    model = LSTM(input_size, hidden_layer_size, output_size).to(device)
    print(f'Model: {model} input_size: {input_size}, hidden_layer_size: {hidden_layer_size}, output_size: {output_size}')
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Training Loop
    train_loss = 0.0
    best_loss = 1e10
    best_epoch = 0
    rec = []
    for epoch in range(num_epochs):
        start = time.time()
        model.train()
        for i, data in enumerate(train_load):
            X_train, y_train = data
            optimizer.zero_grad()
            y_pred = model(X_train)

            loss = criterion(y_pred, y_train)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
        end = time.time()
            
        train_loss = train_loss / len(train_load)

        # Test the model
        test_loss = 0.0
        model.eval()
        with torch.no_grad():
            for i, data in enumerate(test_load):
                X_test, y_test = data
                y_pred = model(X_test)
                loss = criterion(y_pred, y_test)
                test_loss += loss.item()
        test_loss = test_loss / len(test_load)
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Time: {end-start:.2f} sec")
    
        if test_loss < best_loss:
            print(f"Model improved from {best_loss:.4f} to {test_loss:.4f} Saving model...")
            best_loss = test_loss
            best_epoch = epoch + 1

            # Save checkpoint
            checkpoint = {
                'epoch': best_epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': best_loss,
                'input_size': model.input_size,
                'hidden_layer_size': model.hidden_layer_size,
                'output_size': model.output_size
            }
            torch.save(checkpoint, f'./{directory}/model{round(best_loss*10000)}.pth')
            torch.save(checkpoint, f'./{directory}/best_model.pth')

        # Record results
        rec.append({
            'epoch': epoch + 1,
            'train_loss': train_loss,
            'test_loss': test_loss,
        })

    # Write results to csv
    df = pd.DataFrame(rec)
    df.to_csv(f'{directory}/results.csv', index=False)

    print(f"Finihsed training after {num_epochs} epochs. Best loss: {best_loss:.4f} at epoch {best_epoch}")
    


Model: LSTM(
  (lstm): LSTM(7, 48, batch_first=True)
  (linear): Linear(in_features=48, out_features=1, bias=True)
) input_size: 7, hidden_layer_size: 48, output_size: 1


/home/RAI_65011278/miniconda3/envs/binder_env/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/RAI_65011278/miniconda3/envs/binder_env/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([19])) that is different to the input size (torch.Size([19, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10], Train Loss: 0.0019, Test Loss: 0.0009, Time: 7.87 sec
Model improved from 10000000000.0000 to 0.0009 Saving model...


/home/RAI_65011278/miniconda3/envs/binder_env/lib/python3.9/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([21])) that is different to the input size (torch.Size([21, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


AttributeError: 'LSTM' object has no attribute 'input_size'

### **Test the Result**

In [28]:
# Load the trained model
path = 'results/best_model.pth'
checkpoint = torch.load(f'{path}', map_location=device)
print(checkpoint.keys())

# Recreate the model
model = LSTM(checkpoint['input_size'], checkpoint['hidden_layer_size'], output_size=1).to(device)

# Load the model weights
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()  # Set to evaluation mode

print("Model loaded successfully!")

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss', 'input_size', 'hidden_layer_size', 'output_size'])


RuntimeError: Error(s) in loading state_dict for LSTM:
	Missing key(s) in state_dict: "lstm.weight_ih_l1", "lstm.weight_hh_l1", "lstm.bias_ih_l1", "lstm.bias_hh_l1". 

In [ ]:
X_test, y_test = next(iter(test_load))  # Get one batch from test dataset
X_test = X_test.to(device)

# Predict
with torch.no_grad():
    y_pred = model(X_test).cpu().numpy()  # Convert to NumPy for plotting
    # print(f'y_pred:{y_pred.shape} {y_pred}')

# Convert true values to NumPy
y_test = y_test.cpu().numpy()

# Fix the shape by flattening y_pred
y_pred = y_pred.reshape(y_pred.shape[0], -1).mean(axis=1)  # Average across time steps if needed
print(f'y_pred:{y_pred.shape} {y_pred}')
